**Accessing via Huggingface API**

In [ ]:
pip install huggingface_hub

**Accesing Locally**

In [ ]:
pip install torch transformers accelerate sentencepiece

In [ ]:
!git lfs install
!git clone https://huggingface.co/NousResearch/Hermes-3-Llama-3.1-8B

Git LFS initialized.
Cloning into 'Hermes-3-Llama-3.1-8B'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 90 (delta 48), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (90/90), 2.26 MiB | 1.93 MiB/s, done.
Filtering content: 100% (4/4), 2.95 GiB | 17.75 MiB/s, done.
Encountered 3 file(s) that may not have been copied correctly on Windows:
	model-00003-of-00004.safetensors
	model-00001-of-00004.safetensors
	model-00002-of-00004.safetensors

See: `git lfs help smudge` for more details.


In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

############################
# CONFIGURATION
############################
MODEL_PATH = "/content/Hermes-3-Llama-3.1-8B"  # local folder with the model
INPUT_CSV = "/content/Various_Tweets_(2023)_train.csv"                  # your input CSV
OUTPUT_CSV = "/content/Various_Tweets_(2023)_train_with_context.csv"        # output CSV with new 'Context' column

DATASET_DESCRIPTION = (
    "Türkçe Veri Seti, futbol (özellikle bu veri setinde hedeflenen dört Türk futbol takımı Fenerbahçe (FB), Galatasaray (GS), Beşiktaş (BJK) ve Trabzonspor'dur (TS)), sağlık, ekonomi ve siyaset dahil olmak üzere farklı alanlardan gelen tweetlerden oluşmaktadır."
)

############################
# LOAD MODEL LOCALLY
############################
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,  # or bfloat16, etc. adjust as needed
    device_map="auto",         # automatically map to GPU or CPU
)

############################
# GENERATION FUNCTION
############################
def generate_context(text):
    """
    Generates context from the local LLaMA model:
    - No external links
    - Under ~100 words
    """
    # Prompt: no links, under 100 words
    prompt = f"""
Sosyal bir uzman olduğunu varsayarak, aşağıda {DATASET_DESCRIPTION} datasetinden
kısa bir pasaj verilmiştir. Lütfen adım adım düşün, metindeki anahtar kelimeleri çıkar;
yazarın ima ettiği duyguları, retorik araçları vb. analiz et;
ve son olarak yazarın yazı hakkındaki duruşuna dair kısa bir analiz yap, sonuca varmadan analiz sürecini vermeye dikkat et.
Ayrıca açıklama 100 kelimeyi geçmesin ve hiçbir şekilde link içermesin.

Passage: {text}
"""

    # Encode prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=200,        # limit generation
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode the entire output
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Separate out the part after the prompt
    prompt_len = len(tokenizer.encode(prompt))
    response = tokenizer.decode(output_ids[0][prompt_len:], skip_special_tokens=True)

    # Optional: a quick post-process to ensure we don't exceed ~100 words
    #   (model might still exceed if it doesn't follow instructions)
    response_words = response.strip().split()
    if len(response_words) > 100:
        response = " ".join(response_words[:100])

    return response.strip()

############################
# MAIN FLOW
############################
def main():
    print(f"Reading input CSV: {INPUT_CSV}")
    df = pd.read_csv(INPUT_CSV, encoding="utf-8")
    # We'll assume columns "Text" and "Target" exist in the CSV
    # If your file uses different names, update accordingly.
    if "Text" not in df.columns or "Class" not in df.columns:
        raise ValueError("Input CSV must have 'Text' and 'Class' columns.")

    # Generate 'Context' for each row
    contexts = []
    for i, row in df.iterrows():
        text_val = str(row["Text"])
        ctx = generate_context(text_val)
        contexts.append(ctx)
        if (i+1) % 10 == 0:
            print(f"Processed {i+1} rows...")

    df["Context"] = contexts

    # Save the output
    print(f"Saving to {OUTPUT_CSV}")
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print("Done!")

if __name__ == "__main__":
    main()


Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.


Reading input CSV: /content/Various_Tweets_(2023)_train.csv


Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.


...

Processed 2600 rows...


Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.


Processed 2610 rows...


Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128040 for open-end generation.


Saving to /content/Various_Tweets_(2023)_train_with_context.csv
Done!
